In [1]:
from rdflib import Graph

In [3]:
uri = 'http://id.loc.gov/authorities/names/n80002329.rdf'

graph = Graph()
graph.parse(uri)

<Graph identifier=Nb91d2615ccec4a0b9078a1c705271f0f (<class 'rdflib.graph.Graph'>)>

In [4]:
graph.serialize("machado.nt")

<Graph identifier=Nb91d2615ccec4a0b9078a1c705271f0f (<class 'rdflib.graph.Graph'>)>

In [30]:
def GetVariant(authority, graph, obj):
    
  q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  SELECT ?typeVariant ?elementList ?elementValue ?typeElement ?variantLabel 
  WHERE  {{      	
  <{authority}> madsrdf:hasVariant ?variant .
                  ?variant rdf:type ?typeVariant .
                  ?variant rdf:variantLabel ?variantLabel .
                  ?variant madsrdf:elementList ?elementList .
                  ?elementList rdf:rest* ?node .
                    ?node rdf:first ?e .
                    ?e madsrdf:elementValue ?elementValue .
                  ?e rdf:type ?typeElement .
                  FILTER ( ?typeVariant != madsrdf:Variant )
                  }}"""
  result = graph.query(q)
  
  if len(result.bindings) > 0:
    d = {}
    for i in result.bindings:
        bn = i.get('elementList').n3()
        typeVariant = i.get('typeVariant').split("#")[-1]
        elementValue = i.get('elementValue').toPython()
        variantLabel = i.get('variantLabel').toPython()
        tipo = i.get('typeElement').split("#")[-1]
        element = {'type': tipo, 'elementValue': {'value': elementValue}}
        
        if bn in d.keys():
            d[bn]['elementList'].append(element)
        else:
            d[bn] = {'type': typeVariant, 
                     'elementList': [element], 
                     'variantLabel': variantLabel
                     }
    variants = list(d.values())
    obj['hasVariant'] = variants
    return obj
  else:
     return obj
  
GetVariant("http://id.loc.gov/authorities/names/n80002329", graph, {})

{}

In [31]:
authority = "http://id.loc.gov/authorities/names/n80002329"

def GetVariant(authority, graph, obj):
     
  q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?typeVariant ?elementList ?elementValue ?typeElement ?variantLabel 
    WHERE  {{      	
    <{authority}> madsrdf:hasVariant ?variant .
                    ?variant rdf:type ?typeVariant .
                    ?variant madsrdf:variantLabel ?variantLabel .
                    ?variant madsrdf:elementList ?elementList .
                    ?elementList rdf:rest* ?node .
                      ?node rdf:first ?e .
                      ?e madsrdf:elementValue ?elementValue .
                    ?e rdf:type ?typeElement .
                    FILTER ( ?typeVariant != madsrdf:Variant )
                    }}"""
  result = graph.query(q)
  if len(result.bindings) > 0:
    d = {}
    for i in result.bindings:
        bn = i.get('elementList').n3()
        typeVariant = i.get('typeVariant').split("#")[-1]
        elementValue = i.get('elementValue').toPython()
        variantLabel = i.get('variantLabel').toPython()
        tipo = i.get('typeElement').split("#")[-1]
        element = {'type': tipo, 'elementValue': {'value': elementValue}}

        if bn in d.keys():
                d[bn]['elementList'].append(element)
        else:
            d[bn] = {'type': typeVariant, 
                        'elementList': [element], 
                        'variantLabel': variantLabel
                        }
    variants = list(d.values())
    obj['hasVariant'] = variants
    return obj
  else:
     return obj
  
GetVariant(authority, graph, {})

{'hasVariant': [{'type': 'PersonalName',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'Assis, Joaquim Maria Machado de,'}},
    {'type': 'DateNameElement', 'elementValue': {'value': '1839-1908'}}],
   'variantLabel': 'Assis, Joaquim Maria Machado de, 1839-1908'},
  {'type': 'PersonalName',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'Assis, Machado de,'}},
    {'type': 'DateNameElement', 'elementValue': {'value': '1839-1908'}}],
   'variantLabel': 'Assis, Machado de, 1839-1908'},
  {'type': 'PersonalName',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'De Assis, Joaquim Maria Machado,'}},
    {'type': 'DateNameElement', 'elementValue': {'value': '1839-1908'}}],
   'variantLabel': 'De Assis, Joaquim Maria Machado, 1839-1908'},
  {'type': 'PersonalName',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'De Assis, Machado,'}},
    {'type': 'DateNameElement', 'ele

In [29]:
obj = {}
obj['hasVariant'] = variants
obj

{'hasVariant': [{'type': 'PersonalName',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'Assis, Joaquim Maria Machado de,'}},
    {'type': 'DateNameElement', 'elementValue': {'value': '1839-1908'}}],
   'variantLabel': 'Assis, Joaquim Maria Machado de, 1839-1908'},
  {'type': 'PersonalName',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'Assis, Machado de,'}},
    {'type': 'DateNameElement', 'elementValue': {'value': '1839-1908'}}],
   'variantLabel': 'Assis, Machado de, 1839-1908'},
  {'type': 'PersonalName',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'De Assis, Joaquim Maria Machado,'}},
    {'type': 'DateNameElement', 'elementValue': {'value': '1839-1908'}}],
   'variantLabel': 'De Assis, Joaquim Maria Machado, 1839-1908'},
  {'type': 'PersonalName',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'De Assis, Machado,'}},
    {'type': 'DateNameElement', 'ele

In [28]:
obj = GetVariant(authority, graph, {})
obj

{}